In [34]:
import pandas as pd
import plotly.express as px

# Daten laden
df = pd.read_csv('Data/DatenVerkehrsunfälle.csv', sep=',', encoding='utf-8')

# Kombiniere Jahr und Monat in ein Datum
df['AccidentYearMonth'] = pd.to_datetime(
    df['AccidentYear'].astype(str) + '-' + df['AccidentMonth'].astype(str) + '-01'
)

# Daten gruppieren und nach Jahr/Monat und Unfalltyp zählen
df_grouped = df.groupby([df['AccidentYearMonth'], 'AccidentType_de']).size().reset_index(name='Unfälle')

# Gesamtzahl der Unfälle für jeden Unfalltyp berechnen
total_accidents = df.groupby('AccidentType_de').size().reset_index(name='GesamtUnfälle')

# Verbinde die Gesamtunfallzahlen mit den gruppierten Daten
df_grouped = df_grouped.merge(total_accidents, on='AccidentType_de', how='left')

# Daten sortieren: nach Jahr und Monat und danach nach der Häufigkeit der Unfalltypen
df_grouped = df_grouped.sort_values(by=['AccidentYearMonth', 'Unfälle'], ascending=[True, False])

# Unfalltypen nach Häufigkeit sortieren
sorted_columns = df_grouped.groupby('AccidentType_de')['Unfälle'].sum().sort_values(ascending=False).index
df_grouped['AccidentType_de'] = pd.Categorical(df_grouped['AccidentType_de'], categories=sorted_columns, ordered=True)

# Benutzerdefinierte Farbpalette
custom_colors = [
    "#1f77b4",  # Blau
    "#ff7f0e",  # Orange
    "#2ca02c",  # Grün
    "#d62728",  # Rot
    "#9467bd",  # Lila
    "#8c564b",  # Braun
    "#e377c2",  # Pink
    "#7f7f7f",  # Grau
    "#bcbd22",  # Gelb
    "#17becf",  # Türkis
]

# Diagramm mit Plotly erstellen und benutzerdefinierte Farben verwenden
fig = px.area(df_grouped, 
              x='AccidentYearMonth', 
              y='Unfälle', 
              color='AccidentType_de', 
              title='Verkehrsunfälle nach Jahr und Unfalltyp',
              labels={'AccidentYearMonth': 'Jahr', 'Unfälle': 'Anzahl der Unfälle'},
              hover_data={'AccidentYearMonth': True, 'AccidentType_de': True, 'Unfälle': True, 'GesamtUnfälle': True},  # Tooltips mit Gesamtunfällen
              color_discrete_sequence=custom_colors)  # Benutzte Farbpalette

# Linien und Ecken anpassen (abgerundete Ecken für die Flächen)
fig.update_traces(
    line=dict(width=1, dash='solid'),  # Dünnere Linien
    opacity=0.6,  # Transparenz der Flächen
    line_shape='spline'  # Für abgerundete Kanten
)

# Dynamische Tooltip-Hintergrundfarbe basierend auf dem Unfalltyp
fig.update_traces(
    hoverlabel=dict(
        bgcolor=custom_colors[0],  # Hintergrundfarbe basierend auf der ersten Farbe
        font_size=12,
        font_family="Arial",
        font_color="black",
        bordercolor="black"
    )
)

# Tooltip-Design anpassen
fig.update_layout(
    hoverlabel=dict(
        bgcolor="rgba(255, 255, 255, 0.8)",  # Heller Hintergrund
        font_size=12,
        font_family="Arial",
        align="auto",  # 'center' ersetzt durch 'auto'
        bordercolor="black",  # Schwarz für den Rand
        font_color="black"
    ),
    xaxis_title='Jahr',
    yaxis_title='Anzahl der Unfälle',
    legend_title='Unfalltyp',
    template='plotly',
)

# Anzeige des Diagramms
fig.show()
